In [ ]:
%matplotlib inline

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import math
from pymoo.factory import get_performance_indicator
from itertools import product
from palettable.scientific.diverging import Vik_7

pd.set_option('mode.chained_assignment', None)
pd.set_option('display.float_format',lambda x : '%.4f' % x)
plt.style.use(['science','ieee'])


In [ ]:
results_list = [
    "results-database-20220202140407.json"
]

from tinydb import TinyDB, Query

dataset = pd.DataFrame()

for results in results_list:
    database = TinyDB("results/" + results)
    q = Query()
    temp_results = database.all()
    [
        result.update({
            "policy":
            "(s=" + str(int(result['s'])) + ")" if result['algorithm'] == "Heuristic"
            or result['algorithm'] == "HeuristicT"
            or result['algorithm'] == "Augmecon"
            or result['algorithm'] == "WeightedSum" else
            "(nfe=" + str(int(result['nfe'])) + ")"
        }) for result in temp_results
    ]
    temp = pd.DataFrame(temp_results)
    dataset = pd.concat([dataset, temp])
dataset = dataset[[
    "algorithm","number_each_edge_type","s","nfe", "system_accuracies", "points", "constraints", "experiment_time","policy","run"
]]
dataset = dataset.fillna(0)
dataset[['s', 'nfe', 'number_each_edge_type',"run"
         ]] = dataset[['s', 'nfe', 'number_each_edge_type',"run"]].astype(int)

temp_dataset=dataset.copy(deep=True)
temp_dataset['accuracy'] = temp_dataset['constraints'].map(
    lambda x: x['accuracy'][0])
temp_dataset['number'] =    temp_dataset['constraints'].map(lambda x: x['number'][0])
temp_dataset['bandwidth'] = temp_dataset['constraints'].map(
    lambda x: x['bandwidth'][0])
temp_dataset=temp_dataset.groupby(by=["number_each_edge_type"])[['accuracy','number','bandwidth']].min()
refs=temp_dataset.to_records()
ref_points={}
for ref in refs:
    ref=list(ref)
    number_each_edge_type=ref[0]
    ref_point = np.array(ref[1:]) * (-1)
    hv = get_performance_indicator("hv", ref_point=ref_point)
    kv=dict(ref=ref_point,hv=hv)
    ref_points.update({number_each_edge_type: kv})

dataset = dataset.drop(columns=["constraints"])

ref_points

In [ ]:
time_results = dataset[[
    'policy', 'number_each_edge_type', 'experiment_time', 'algorithm',"run"
]]

time_results.sort_values(['experiment_time'], ascending=[False])
time_results.reset_index(inplace=True)
time_results = time_results.rename(columns={"experiment_time": "Time"})
time_results

In [ ]:
df=dataset.copy(deep=True)
heuristic_dataset=[]
if len(df.index)>0:
    df = df[['points', 'policy', 'number_each_edge_type', 'algorithm','run']]
    df=list(df.to_dict('records'))
    for item in df:
        points=item.pop('points')
        for point in points:
            point.update(item)
            heuristic_dataset.append(point)

df=pd.DataFrame(heuristic_dataset).drop_duplicates(subset=None, keep='first', inplace=False)
# df.head()
hv_results = []
for algorithm in df['algorithm'].unique():
    filter0= df['algorithm']==algorithm
    temp_df=df.where(filter0)
    for policy, number,run in product(temp_df['policy'].unique(),
                                  temp_df['number_each_edge_type'].unique(),
                                  temp_df['run'].unique()):
        filter1 = temp_df['policy'] == policy
        filter2 = temp_df['number_each_edge_type'] == number
        filter3 = temp_df['run'] == run
        temp = temp_df[[
            'served', 'average_accuracy', 'bandwidth_reserved', 'policy',
            'number_each_edge_type', 'algorithm','run'
        ]].where(filter1 & filter2 & filter3).dropna(axis=0, how='any')
        if temp.size > 0:
            data = temp[['average_accuracy', 'served', 'bandwidth_reserved']]
            result = {
                'policy': policy,
                'number_each_edge_type': number,
                'algorithm': algorithm,
                'run':run
            }
            front = data.values
            hv=ref_points[number]['hv']
            result.update({
                "HV": hv.do(-front)
            })
            hv_results.append(result)
hv_results = pd.DataFrame(hv_results)
hv_results[['number_each_edge_type','run',
         ]] = hv_results[['number_each_edge_type','run']].astype(int)

hv_results

In [ ]:
fig, axes = plt.subplots(1,
                         2,
                         gridspec_kw={
                             'width_ratios': [1, 1],
                             "wspace": 0.25,
                         },
                         sharex=True,
                         sharey=False,
                         figsize=(10, 2))

temp_hv_results=hv_results.copy(deep=True)
temp_hv_results = temp_hv_results.rename(columns={'number_each_edge_type':"number"})
temp_hv_results['number'] = temp_hv_results['number']*3
temp_hv_results['algorithm'] = temp_hv_results['algorithm'].str.replace(
    "Augmecon", "ECON")
temp_hv_results['algorithm'] = temp_hv_results['algorithm'].str.replace(
    "WeightedSum", "WSUM")
temp_hv_results['algorithm'] = temp_hv_results['algorithm'].str.replace(
    "HeuristicT", "WCH-T")
temp_hv_results['algorithm'] = temp_hv_results['algorithm'].str.replace(
    "Heuristic", "WCH")
temp_hv_results['algorithm'] = temp_hv_results['algorithm'].str.replace(
    "NSGAII", "EVOL")

g1 = sns.barplot(x='number',
                 y='HV',
                 hue_order=["EVOL", "ECON", "WSUM", "WCH-T", "WCH"],
                 hue='algorithm',
                 data=temp_hv_results,
                 edgecolor=".1",
                 linewidth=1,
                 palette=Vik_7.mpl_colors[1:6],
                 ax=axes[0])
g1.get_legend().set_visible(False)
g1.set_ylim(200,3000)

g1right = g1.twinx()
g1right.set_ylim(200, 3000)

temp_time_results = time_results.copy(deep=True)
temp_time_results = temp_time_results.rename(
    columns={'number_each_edge_type': "number"})
temp_time_results['number'] = temp_time_results['number']*3
temp_time_results['algorithm'] = temp_time_results['algorithm'].str.replace(
    "Augmecon", "ECON")
temp_time_results['algorithm'] = temp_time_results['algorithm'].str.replace(
    "WeightedSum", "WSUM")
temp_time_results['algorithm'] = temp_time_results['algorithm'].str.replace(
    "HeuristicT", "WCH-T")
temp_time_results['algorithm'] = temp_time_results['algorithm'].str.replace(
    "Heuristic", "WCH")
temp_time_results['algorithm'] = temp_time_results['algorithm'].str.replace(
    "NSGAII", "EVOL")

g2 = sns.barplot(x='number',
                 y='Time',
                 hue_order=["EVOL","ECON","WSUM","WCH-T","WCH"],
                 hue='algorithm',
                 data=temp_time_results,
                 edgecolor=".1",
                 linewidth=1,
                 palette=Vik_7.mpl_colors[1:6],
                 ax=axes[1])
g2.set_yscale("log")
g2.get_legend().set_bbox_to_anchor((1.0, 1.0))
g2.set_ylim(80, 14000)
g2right = g2.twinx()
g2right.set_yscale("log")
g2right.set_ylim(80, 14000)


g1.yaxis.label.set_text('\\textbf{ Hypervolume }')
g1.xaxis.label.set_text('\\textbf{\# of Edge Nodes}')
g2.yaxis.label.set_text('\\textbf{ Time Cost (sec.)}')
g2.xaxis.label.set_text('\\textbf{\# of Edge Nodes}')
g1.xaxis.set_tick_params(labelsize='small')
g1.yaxis.set_tick_params(labelsize='small')
g2.xaxis.set_tick_params(labelsize='small')
g2.yaxis.set_tick_params(labelsize='small')
g2.get_legend().set_visible(False)

g1right.yaxis.label.set_text('\\textbf{ Hypervolume }')
g2right.yaxis.label.set_text('\\textbf{ Time Cost (sec.)}')
g1right.yaxis.label.set_color("white")
g2right.yaxis.label.set_color("white")
g1right.yaxis.set_tick_params(labelsize='small',labelcolor="white")
g2right.yaxis.set_tick_params(labelsize='small',labelcolor="white")
g1right.tick_params(axis='y', which="major", width=1)
g2right.tick_params(axis='y', which="major", width=1)

g1.spines['left'].set_linewidth(1)
g1.spines['right'].set_linewidth(1)
g1.spines['top'].set_linewidth(1)
g1.spines['bottom'].set_linewidth(1)
g2.spines['left'].set_linewidth(1)
g2.spines['right'].set_linewidth(1)
g2.spines['top'].set_linewidth(1)
g2.spines['bottom'].set_linewidth(1)

hatches = [r'', r'//////', r'', r'\\\\\\', r'']
for ax in axes.flatten():
    for i, thisbar in enumerate(ax.patches):
        thisbar.set_hatch(hatches[math.floor(i / 3)])

labels = [
    "\\textsf{EVOL}",
    "\\textsf{ECON}",
    "\\textsf{WSUM}",
    "\\textsf{WCH-T}",
    "\\textsf{WCH}",
]

handles,_ = g1.get_legend_handles_labels()

g1.legend(
    title='',
    labels=labels,
    handles=handles,
    fontsize='small',
    handlelength=1.5,
    handletextpad=1,
    labelspacing=0.5,
    loc="upper left",
    bbox_to_anchor=(0.05, 0.95)
)

for ax in axes.flatten():
    ax.spines['left'].set_linewidth(1)
    ax.spines['right'].set_linewidth(1)
    ax.spines['top'].set_linewidth(1)
    ax.spines['bottom'].set_linewidth(1)
    ax.tick_params(axis='x', which="major", width=1)
    ax.tick_params(axis='y', which="major", width=1)

plt.savefig("figures/algorithm.pdf", dpi=300)